In [1]:
import pandas as pd
import numpy as np
import scipy.io
import math
from collections import Counter
import copy
import warnings
import tensorflow as tf
from tensorflow.keras.layers import *
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import BatchNormalization
warnings.filterwarnings("ignore")

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [2]:

def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):

            pretty(value, indent + 2)
            print('-----------------------')
        else:
            print('\t' * (indent + 2) + str(value))
            print(" ")
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
def extractSubject(name):
    ex1Path = 'DB1/' + name + '/' + name + '_A1_E1.mat'
    print(ex1Path)
    ex1 = scipy.io.loadmat(ex1Path)
    emg = ex1['emg']
    EMGdf = pd.DataFrame.from_dict(emg)
    stimulus = ex1['stimulus']

    ex2Path = 'DB1/' + name + '/' + name + '_A1_E2.mat'
    ex2 = scipy.io.loadmat(ex2Path)
    emg2 = ex2['emg']
    EMGdf2 = pd.DataFrame.from_dict(emg2)
    stimulus2 = ex2['stimulus']

    ex3Path = 'DB1/' + name + '/' + name + '_A1_E3.mat'
    ex3 = scipy.io.loadmat(ex3Path)
    emg3 = ex3['emg']
    EMGdf3 = pd.DataFrame.from_dict(emg3)
    stimulus3 = ex3['stimulus']

    Movements = {}
    for m in range(1, 53):
        if (m < 13):
            movementIndices = np.where(stimulus == m)[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf
        elif (m < 30):
            movementIndices = np.where(stimulus2 == (m - 12))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf2
        else:
            movementIndices = np.where(stimulus3 == (m - 29))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf3

        Repetitions = {}
        for r in range(1, 7):
            startIndex = repetitions[r - 1][0]
            LastIndex = repetitions[r - 1][len(repetitions[r - 1]) - 1]
            df = EMG.iloc[startIndex:LastIndex, 0:10]
            df.reset_index(drop=True, inplace=True)
            Repetitions["R{0}".format(r)] = df
        Movements["Movement{0}".format(m)] = Repetitions
    return Movements
def Average(lst):
    return sum(lst) / len(lst)


In [101]:

columns = {'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'Train','Movement'}
X_train = []
y_train = []
X_test = []
y_test = []
for s in range(1,28):
    subject = 'S' + str(s)
    dff = pd.DataFrame.from_dict(extractSubject(subject))
    for m in range(1,53):
        M = dff['Movement'+str(m)]
        for r in range(1,7): 
            r_index = "R" + str(r)
            rep = M[r_index]
            if (r in [1, 3, 4, 6]):
                train = True
            else:
                train = False
            i=0    
            for x in range(0, len(rep), 6):
                entry = rep.iloc[x:x+15, :]
                entry.columns = {'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10'}
                entry.reset_index(drop=True, inplace=True)
                matrix = entry.to_numpy()
                if matrix.shape[0]==15:
                    if(train):
                        X_train.append(matrix)
                        y_train.append(m)
                    else :
                        X_test.append(matrix)
                        y_test.append(m)  
                    i+=1
                       


DB1/S1/S1_A1_E1.mat
DB1/S2/S2_A1_E1.mat
DB1/S3/S3_A1_E1.mat
DB1/S4/S4_A1_E1.mat
DB1/S5/S5_A1_E1.mat
DB1/S6/S6_A1_E1.mat
DB1/S7/S7_A1_E1.mat
DB1/S8/S8_A1_E1.mat
DB1/S9/S9_A1_E1.mat
DB1/S10/S10_A1_E1.mat
DB1/S11/S11_A1_E1.mat
DB1/S12/S12_A1_E1.mat
DB1/S13/S13_A1_E1.mat
DB1/S14/S14_A1_E1.mat
DB1/S15/S15_A1_E1.mat
DB1/S16/S16_A1_E1.mat
DB1/S17/S17_A1_E1.mat
DB1/S18/S18_A1_E1.mat
DB1/S19/S19_A1_E1.mat
DB1/S20/S20_A1_E1.mat
DB1/S21/S21_A1_E1.mat
DB1/S22/S22_A1_E1.mat
DB1/S23/S23_A1_E1.mat
DB1/S24/S24_A1_E1.mat
DB1/S25/S25_A1_E1.mat
DB1/S26/S26_A1_E1.mat
DB1/S27/S27_A1_E1.mat


In [121]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
print("X Train shape: ", X_train.shape)
print("X Test shape: ", X_test.shape)
print("Y Train shape: ", y_train.shape)
print("Y TEST shape: ", y_test.shape)
print('------------------')
X_train = np.array(X_train).reshape(X_train.shape[0],15,10)
X_test = np.array(X_test).reshape(X_test.shape[0],15,10)
# y_train = np.array(y_train).astype('float32').reshape(X_train.shape[0],1)
# y_test = np.array(y_test).astype('float32').reshape(X_test.shape[0],1)

# y_train= tf.one_hot(y_train,52)
# y_test= tf.one_hot(y_test,52)
# y_test = np.array(y_test).astype('float32').reshape(X_test.shape[0],52)
# y_train = np.array(y_train).astype('float32').reshape(X_train.shape[0],52)

# print("X Train shape: ", X_train.shape)
# print("X Test shape: ", X_test.shape)
# print("Y Train shape: ", y_train.shape)

X Train shape:  (472803, 15, 10)
X Test shape:  (236439, 15, 10)
Y Train shape:  (472803, 52)
Y TEST shape:  (236439, 52)
------------------


In [133]:
input = Input(shape =(15,10,1))

X = ZeroPadding2D((0,4))(input)
X = Conv2D(32, (1, 10), padding='valid', kernel_initializer='glorot_normal', name='b1_conv2d_32_1x10')(X)
X = Activation('relu', name='b1_relu')(X)
X = Dropout(0.15, name='b1_dropout')(X)
X = BatchNormalization()(X)

X = ZeroPadding2D((1,1))(X)
X = Conv2D(32, (3, 3), padding='valid', kernel_initializer='glorot_normal', name='b2_conv2d_32_3x3')(X)
X = Activation('relu', name='b2_relu')(X)
X = Dropout(0.15, name='b2_dropout')(X)
X = MaxPooling2D((3,3), strides = (3,3), name='b2_pool')(X)
X = BatchNormalization()(X)

X = ZeroPadding2D((2,2))(X)
X = Conv2D(64, (5, 5), padding='valid', kernel_initializer='glorot_normal', name='b3_conv2d_64_5x5')(X)
X = Activation('relu', name='b3_relu')(X)
X = Dropout(0.15, name='b3_dropout')(X)
X = MaxPooling2D((3,3), strides = (3,3), name='b3_pool')(X)
X = BatchNormalization()(X)

X = ZeroPadding2D((2,0))(X)
X = Conv2D(64, (5, 1), padding='valid', kernel_initializer='glorot_normal', name='b4_conv2d_64_5x1')(X)
X = Activation('relu', name='b4_relu')(X)
X = Dropout(0.15, name='b4_dropout')(X)
X = BatchNormalization()(X)

X = Conv2D(52, (1, 1), padding='same',  kernel_initializer='glorot_normal', name='b5_conv2d_{}_1x1'.format(52))(X)
X = Activation('softmax', name='b5_soft')(X)
X = Reshape((-1,), name='b5_reshape')(X)

model = Model(input, X)
model.compile(optimizer=SGD(learning_rate=0.05), loss="categorical_crossentropy", metrics=['accuracy'])
print(X_train.shape)

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
print(model.evaluate(X_test, y_test))

model.summary()

(472803, 15, 10)
Epoch 1/10
14776/14776 [==============================] - 104s 7ms/step - loss: 3.3452 - accuracy: 0.1251 - val_loss: 3.2299 - val_accuracy: 0.1499
Epoch 2/10
14776/14776 [==============================] - 108s 7ms/step - loss: 3.1123 - accuracy: 0.1719 - val_loss: 3.1406 - val_accuracy: 0.1751
Epoch 3/10
14776/14776 [==============================] - 111s 7ms/step - loss: 3.0347 - accuracy: 0.1892 - val_loss: 3.1585 - val_accuracy: 0.1661
Epoch 4/10
14776/14776 [==============================] - 109s 7ms/step - loss: 2.9923 - accuracy: 0.1991 - val_loss: 3.0335 - val_accuracy: 0.2096
Epoch 5/10
14776/14776 [==============================] - 108s 7ms/step - loss: 2.9613 - accuracy: 0.2064 - val_loss: 3.0911 - val_accuracy: 0.1860
Epoch 6/10
14776/14776 [==============================] - 108s 7ms/step - loss: 2.9403 - accuracy: 0.2101 - val_loss: 3.0112 - val_accuracy: 0.2061
Epoch 7/10
14776/14776 [==============================] - 108s 7ms/step - loss: 2.9219 - accura

In [128]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
class_labels = np.argmax(y_pred, axis=1) 
y_df = pd.DataFrame(class_labels)
y_df.to_csv('y_pred.csv')
class_labels_2 = np.argmax(y_test, axis=1) 
print(class_labels)
print(class_labels_2)
y_test_new = [most_frequent(class_labels_2[x:x + 83]) for x in range(0, len(class_labels_2), 83)]
y_predicted_new = [most_frequent(class_labels[x:x + 83]) for x in range(0, len(class_labels), 83)]
accuracy_modified = accuracy_score(y_test_new, y_predicted_new)
print(accuracy_modified)

[20 20 20 ... 36 36 36]
[1 1 1 ... 0 0 0]
0.5178843226788432


In [112]:
X_train.shape
# y_train.shape
# y_test.shape

(472803, 15, 10)

In [ ]:
model.history['accuracy']
model.history['loss']